In [1]:
import tkinter as tk
from tkinter import messagebox
import json
import os
import uuid
from datetime import datetime

def create_account(name, initial_balance=0.0):
    account_id = str(uuid.uuid4())
    account = {"id": account_id, "name": name, "balance": initial_balance, "transactions": []}
    if not os.path.exists("accounts.json"):
        with open("accounts.json", "w") as file:
            json.dump({}, file)
    with open("accounts.json", "r+") as file:
        data = json.load(file)
        data[account_id] = account
        file.seek(0)
        json.dump(data, file, indent=4)
    return account

def load_account(account_id):
    if os.path.exists("accounts.json"):
        with open("accounts.json", "r") as file:
            data = json.load(file)
            if account_id in data:
                return data[account_id]
    raise ValueError("Account not found.")

def save_account(account):
    with open("accounts.json", "r+") as file:
        data = json.load(file)
        data[account["id"]] = account
        file.seek(0)
        json.dump(data, file, indent=4)

def deposit(account, amount):
    if amount > 0:
        account["balance"] += amount
        transaction = {"type": "Deposit", "amount": amount, "balance": account["balance"], "date": str(datetime.now())}
        account["transactions"].append(transaction)
        save_account(account)
    else:
        raise ValueError("Deposit amount must be positive.")

def withdraw(account, amount):
    if amount > 0:
        if account["balance"] >= amount:
            account["balance"] -= amount
            transaction = {"type": "Withdrawal", "amount": amount, "balance": account["balance"], "date": str(datetime.now())}
            account["transactions"].append(transaction)
            save_account(account)
        else:
            raise ValueError("Insufficient balance.")
    else:
        raise ValueError("Withdrawal amount must be positive.")

def transfer(sender_account, receiver_account_id, amount):
    if amount > 0:
        if sender_account["balance"] >= amount:
            receiver_account = load_account(receiver_account_id)
            sender_account["balance"] -= amount
            receiver_account["balance"] += amount

            sender_transaction = {"type": "Transfer Out", "amount": amount, "balance": sender_account["balance"], "date": str(datetime.now()), "to": receiver_account_id}
            receiver_transaction = {"type": "Transfer In", "amount": amount, "balance": receiver_account["balance"], "date": str(datetime.now()), "from": sender_account["id"]}

            sender_account["transactions"].append(sender_transaction)
            receiver_account["transactions"].append(receiver_transaction)

            save_account(sender_account)
            save_account(receiver_account)
        else:
            raise ValueError("Insufficient balance.")
    else:
        raise ValueError("Transfer amount must be positive.")

def check_balance(account):
    return account["balance"]

def print_statement(account):
    statement = f"Account statement for {account['name']} (ID: {account['id']}):\n"
    for transaction in account["transactions"]:
        statement += f"- {transaction['date']} | {transaction['type']}: {transaction['amount']}, Balance: {transaction['balance']}\n"
    return statement

def create_gui():
    current_account = None

    def handle_create_account():
        nonlocal current_account
        name = account_name_entry.get()
        try:
            current_account = create_account(name)
            messagebox.showinfo("Success", f"Account for {name} created successfully!\nAccount ID: {current_account['id']}")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def handle_load_account():
        nonlocal current_account
        account_id = account_id_entry.get()
        try:
            current_account = load_account(account_id)
            messagebox.showinfo("Success", f"Account for {current_account['name']} loaded successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def handle_deposit():
        try:
            amount = float(deposit_entry.get())
            deposit(current_account, amount)
            messagebox.showinfo("Success", f"Deposited {amount} successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def handle_withdraw():
        try:
            amount = float(withdraw_entry.get())
            withdraw(current_account, amount)
            messagebox.showinfo("Success", f"Withdrew {amount} successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def handle_transfer():
        try:
            receiver_id = receiver_account_entry.get()
            amount = float(transfer_entry.get())
            transfer(current_account, receiver_id, amount)
            messagebox.showinfo("Success", f"Transferred {amount} successfully to {receiver_id}!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def handle_check_balance():
        try:
            balance = check_balance(current_account)
            messagebox.showinfo("Balance", f"Current balance: {balance}")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def handle_print_statement():
        try:
            statement = print_statement(current_account)
            messagebox.showinfo("Statement", statement)
        except Exception as e:
            messagebox.showerror("Error", str(e))

    root = tk.Tk()
    root.title("Fully Advanced Banking System")

    tk.Label(root, text="Account Name:").grid(row=0, column=0, padx=10, pady=5)
    account_name_entry = tk.Entry(root)
    account_name_entry.grid(row=0, column=1, padx=10, pady=5)
    tk.Button(root, text="Create Account", command=handle_create_account).grid(row=0, column=2, padx=10, pady=5)

    tk.Label(root, text="Account ID:").grid(row=1, column=0, padx=10, pady=5)
    account_id_entry = tk.Entry(root)
    account_id_entry.grid(row=1, column=1, padx=10, pady=5)
    tk.Button(root, text="Load Account", command=handle_load_account).grid(row=1, column=2, padx=10, pady=5)

    tk.Label(root, text="Deposit Amount:").grid(row=2, column=0, padx=10, pady=5)
    deposit_entry = tk.Entry(root)
    deposit_entry.grid(row=2, column=1, padx=10, pady=5)
    tk.Button(root, text="Deposit", command=handle_deposit).grid(row=2, column=2, padx=10, pady=5)

    tk.Label(root, text="Withdraw Amount:").grid(row=3, column=0, padx=10, pady=5)
    withdraw_entry = tk.Entry(root)
    withdraw_entry.grid(row=3, column=1, padx=10, pady=5)
    tk.Button(root, text="Withdraw", command=handle_withdraw).grid(row=3, column=2, padx=10, pady=5)

    tk.Label(root, text="Transfer Amount:").grid(row=4, column=0, padx=10, pady=5)
    transfer_entry = tk.Entry(root)
    transfer_entry.grid(row=4, column=1, padx=10, pady=5)
    tk.Label(root, text="Receiver Account ID:").grid(row=4, column=2, padx=10, pady=5)
    receiver_account_entry = tk.Entry(root)
    receiver_account_entry.grid(row=4, column=3, padx=10, pady=5)
    tk.Button(root, text="Transfer", command=handle_transfer).grid(row=4, column=4, padx=10, pady=5)

    tk.Button(root, text="Check Balance", command=handle_check_balance).grid(row=5, column=0, columnspan=2, pady=5)
    tk.Button(root, text="Print Statement", command=handle_print_statement).grid(row=5, column=2, columnspan=2, pady=5)

    root.mainloop()

create_gui()

